In [22]:
import json
import sys
from zipfile import ZipFile

import os
import hashlib

import datetime
import pytz

import telegram

from pprint import pprint
from modules.translator import Translator
from modules.sunless import RecursiveProcessor

OUTPUT_PATH = 'output'
ORIGIN_PATH = 'entities'
TRANS_PATH = 'entities_ko'
STATUS_PATH = 'status'

CONFIG_FILE = 'config.json'


SHEET_DOWNLOAD_PATH = "status"
SHEET_ORIGINAL_PATH = "entities"

SHEET_VALUE_CATES = [
    'events',
    'qualities'
]

In [23]:
from modules.config import config

In [34]:
cate = SHEET_VALUE_CATES[0]
    
with open(os.path.join(ORIGIN_PATH, "%s.json" % cate)) as f:
    origin = json.load(f)

with open(os.path.join(SHEET_DOWNLOAD_PATH, "%s.json" % cate)) as f:
    google = json.load(f)


In [35]:
class Flatter(RecursiveProcessor):
        
    def process(self, root):
        self.result_dict = {}
        self.crash_list = []
        succ, failed = super(Flatter, self).process(root)
        print("succ", succ, "failed", failed)
        return self.result_dict, self.crash_list

    def _process_node(self, node, parentName):
        #node_id = "%s/%s" % (parentName, node['Id'])
        node_id = node['Id']
        
        if node_id in self.result_dict:
            self.result_dict[node_id].append(
                {
                    'Id': node['Id'],
                    'Name': node.get('Name', None),
                    'Teaser': node.get('Teaser', None),
                    'Description': node.get('Description', None),
                    'keys': list(node.keys())
                }
            )
            self.crash_list.append(node_id)
#             print("===================================")
#             print("Crash ", node_id)
#             print(self.result_dict[node_id])
#             print("-----------------------------")
#             print(node)
            return False
        
        self.result_dict[node_id] = [{
            'Id': node['Id'],
            'Name': node.get('Name', None),
            'Teaser': node.get('Teaser', None),
            'Description': node.get('Description', None),
            'keys': list(node.keys())
        }]

        return True

In [36]:
flatter = Flatter()
origin, crs=flatter.process(origin)

succ 10433 failed 220


In [56]:
missing_ids = []
notmatched_ids = []
matched_ids = []

duplicated_ids = []

for k, v in origin.items():
    sk = str(k)
    
    if len(v) > 1:
        duplicated_ids.append(k)

    if sk not in google:
        #print("%s is missing" % k)
        missing_ids.append(k)
    else:
        match = google[sk]
            
        for i, o in enumerate(v):
            
            if o['Name'] != match['Name_origin'] and o['Name'] != "'%s" % match['Name_origin']:
                notmatched_ids.append("%s-%s" % (k, i))
            else:
                matched_ids.append("%s-%s" % (k, i))
    
print(len(missing_ids), len(notmatched_ids), len(matched_ids), len(duplicated_ids))

687 226 9740 220


In [48]:
for idx in notmatched_ids:
    d = '+' if idx in matched_ids else ''
    print(idx, d, origin[int(idx[:-2])][int(idx[-2:])]['Name'], '//' , google[idx[:-2]]['Name_origin'])

177542-0  "Yes. There was a certain report." // Visit the Cock and Magpie
146187-0   // Help Maybe's Daughter search for her mother
182205-1  Guard the Sprightly Visionary's headquarters // An interlude
146919-0   // Recall other places where there is no sky
150910-0  Deliver your smuggled souls // A precarious embrace
138515-0  None // A continuing mystery...
139752-0  Shipwrecked! // Watch a Visager
148311-1  Gather Strategic Information for the Admiralty // Those are pearls
149343-0  Climb higher, see farther // An accommodation with London
149344-0  Something distant // An understanding with the Presbyterate
149346-1  No one // Something bright
149347-0  The high airs // Knowledge is Power
149348-0  Look north, look east // Coffee is Power
149349-0  New vistas // Wealth is Power
146899-1  Assemble a Port Report // A gloomy inspiration
146902-1  Descend into the basement with sunlight // A joyful noise
146901-0  A gloomy inspiration // Sit down a moment
146904-0  A joyful noise // P

In [52]:
origin[146187][1]['Name']

"Help Maybe's Daughter search for her mother"